In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures

In [2]:
X_train = pd.read_csv('ds6-league-of-legends/participants_train.csv')
X_test = pd.read_csv('ds6-league-of-legends/participants_test.csv')
y_train = pd.read_csv('ds6-league-of-legends/train_winners.csv')
submission = pd.read_csv('ds6-league-of-legends/submission.csv')
champion_mastery = pd.read_csv('ds6-league-of-legends/champion_mastery.csv')
Position = pd.read_csv('ds6-league-of-legends/teamPositions.csv')
X_train_timeline = pd.read_csv('ds6-league-of-legends/train_timeline_values.csv')
Y_test_timeline = pd.read_csv('ds6-league-of-legends/test_timeline_values.csv')
champions = pd.read_json('ds6-league-of-legends/champion.json')

In [3]:
#creating a copy of the X data to avoid corruption in the dataset
X_train_original = X_train
y_train_original = y_train
X_test_original = X_test

# this is to extract the columns that is needed for training
y_train = y_train['winner']

In [4]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_train = pd.merge(X_train_original, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_train = X_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_train = (
    X_train
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)


#X_train = X_train.groupby('matchId')[variables].sum().reset_index(drop = True)


X_train

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,0,100,860,30.0,14146,17159,316,409,6268,11867,21635,375,940,34
1,0,200,532,30.0,17539,18773,289,579,6040,13396,21385,943,374,35
2,1,100,570,33.0,17662,17770,255,511,5636,11445,16807,1272,959,34
3,1,200,567,25.0,14816,18028,316,463,6414,8987,22478,959,1272,34
4,2,100,619,25.0,16374,16837,292,556,5913,14196,23323,195,483,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,7997,200,464,29.0,16571,15721,352,534,5940,13150,24331,1715,1248,33
15996,7998,100,689,26.0,15543,15853,277,411,6585,10665,21440,211,346,32
15997,7998,200,1165,32.0,18311,18654,278,553,6067,15595,17865,347,211,36
15998,7999,100,991,26.0,17058,19319,304,517,5805,10454,17554,228,591,35


In [5]:
team1 = X_train[X_train['teamId'] == 100]
team1

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,0,100,860,30.0,14146,17159,316,409,6268,11867,21635,375,940,34
2,1,100,570,33.0,17662,17770,255,511,5636,11445,16807,1272,959,34
4,2,100,619,25.0,16374,16837,292,556,5913,14196,23323,195,483,32
6,3,100,515,28.0,19247,17562,348,531,6240,16263,23900,1020,890,34
8,4,100,747,28.0,16776,18264,304,453,6292,11343,21601,163,800,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15990,7995,100,772,30.0,19009,18803,290,531,6395,16252,20485,94,722,36
15992,7996,100,533,28.0,16413,17298,426,426,6367,14939,19961,309,1015,33
15994,7997,100,567,29.0,17745,17090,302,556,5918,16493,18230,1249,1715,34
15996,7998,100,689,26.0,15543,15853,277,411,6585,10665,21440,211,346,32


In [6]:
team2 = X_train[X_train['teamId'] == 200]
team2

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
1,0,200,532,30.0,17539,18773,289,579,6040,13396,21385,943,374,35
3,1,200,567,25.0,14816,18028,316,463,6414,8987,22478,959,1272,34
5,2,200,658,24.0,17347,17467,317,527,6500,15374,24127,483,195,34
7,3,200,599,31.0,15145,16196,301,530,5760,13223,23420,890,1020,32
9,4,200,352,21.0,15385,17194,300,583,5817,12364,20043,800,163,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15991,7995,200,898,30.0,14873,16977,304,509,5916,12284,23980,723,94,34
15993,7996,200,873,30.0,16140,17927,315,573,6110,12364,23494,1015,309,34
15995,7997,200,464,29.0,16571,15721,352,534,5940,13150,24331,1715,1248,33
15997,7998,200,1165,32.0,18311,18654,278,553,6067,15595,17865,347,211,36


In [7]:
X_teams = pd.merge(team1, team2, how='outer', on=['matchId'])

X_teams

,matchId,teamId_x,summonerLevel_x,championLevel_x,final_gold_x,final_xp_x,final_armor_x,final_atkdmg_x,final_health_x,final_dmgdone_x,...,final_gold_y,final_xp_y,final_armor_y,final_atkdmg_y,final_health_y,final_dmgdone_y,final_dmgtaken_y,final_truedmgdone_y,final_truedmgtaken_y,final_lvl_y
0,0,100,860,30.0,14146,17159,316,409,6268,11867,...,17539,18773,289,579,6040,13396,21385,943,374,35
1,1,100,570,33.0,17662,17770,255,511,5636,11445,...,14816,18028,316,463,6414,8987,22478,959,1272,34
2,2,100,619,25.0,16374,16837,292,556,5913,14196,...,17347,17467,317,527,6500,15374,24127,483,195,34
3,3,100,515,28.0,19247,17562,348,531,6240,16263,...,15145,16196,301,530,5760,13223,23420,890,1020,32
4,4,100,747,28.0,16776,18264,304,453,6292,11343,...,15385,17194,300,583,5817,12364,20043,800,163,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,100,772,30.0,19009,18803,290,531,6395,16252,...,14873,16977,304,509,5916,12284,23980,723,94,34
7996,7996,100,533,28.0,16413,17298,426,426,6367,14939,...,16140,17927,315,573,6110,12364,23494,1015,309,34
7997,7997,100,567,29.0,17745,17090,302,556,5918,16493,...,16571,15721,352,534,5940,13150,24331,1715,1248,33
7998,7998,100,689,26.0,15543,15853,277,411,6585,10665,...,18311,18654,278,553,6067,15595,17865,347,211,36


In [8]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_test = pd.merge(X_test_original, Y_test_timeline, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_test = (
    X_test
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

#X_test = X_test.groupby('matchId')[variables].sum().reset_index(drop = True)

X_test

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,8000,100,788,31.0,17313,18099,288,510,6304,11079,18916,604,520,35
1,8000,200,685,28.0,16933,17117,272,508,5731,12235,21465,521,604,33
2,8001,100,685,31.0,17492,16756,264,499,5584,14834,19547,176,160,33
3,8001,200,539,26.0,17533,16527,285,424,6141,14326,21925,160,176,33
4,8002,100,612,29.0,16206,16640,303,494,5869,15843,24991,338,376,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,9997,200,574,26.0,17129,18888,269,507,5629,14181,18033,523,394,35
3996,9998,100,556,30.0,18353,18298,344,625,6043,13838,20754,679,70,35
3997,9998,200,655,30.0,15134,16823,293,495,6042,12048,23098,70,679,34
3998,9999,100,620,26.0,14561,17572,324,436,6125,12054,20404,939,211,33


In [9]:
X_test1 =  X_test[X_test['teamId'] == 100]
X_test1

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,8000,100,788,31.0,17313,18099,288,510,6304,11079,18916,604,520,35
2,8001,100,685,31.0,17492,16756,264,499,5584,14834,19547,176,160,33
4,8002,100,612,29.0,16206,16640,303,494,5869,15843,24991,338,376,33
6,8003,100,707,30.0,17784,16759,293,565,6250,17100,23946,1498,604,33
8,8004,100,611,27.0,16341,18377,267,454,5802,13477,17305,687,406,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,9995,100,905,35.0,15440,17688,279,414,6072,13331,19797,1835,743,34
3992,9996,100,560,28.0,19123,18273,291,541,6542,13467,23297,1566,76,35
3994,9997,100,510,25.0,14719,17211,282,512,5914,7891,23464,394,522,35
3996,9998,100,556,30.0,18353,18298,344,625,6043,13838,20754,679,70,35


In [10]:
X_test2 =  X_test[X_test['teamId'] == 200]
X_test2

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
1,8000,200,685,28.0,16933,17117,272,508,5731,12235,21465,521,604,33
3,8001,200,539,26.0,17533,16527,285,424,6141,14326,21925,160,176,33
5,8002,200,653,25.0,17480,16518,268,509,6248,17065,23649,376,338,33
7,8003,200,1339,23.0,18126,17173,402,521,6314,16230,25839,604,1497,34
9,8004,200,660,27.0,16993,17113,324,523,6612,11488,20156,406,687,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,9995,200,613,25.0,15060,17845,285,449,6519,11433,21529,743,1835,35
3993,9996,200,697,30.0,17433,18545,328,566,6238,14474,23791,76,1566,36
3995,9997,200,574,26.0,17129,18888,269,507,5629,14181,18033,523,394,35
3997,9998,200,655,30.0,15134,16823,293,495,6042,12048,23098,70,679,34


In [29]:
y_teams_test = pd.merge(X_test1, X_test2, how='outer', on=['matchId'])

y_teams_test

,matchId,teamId_x,summonerLevel_x,championLevel_x,final_gold_x,final_xp_x,final_armor_x,final_atkdmg_x,final_health_x,final_dmgdone_x,...,final_gold_y,final_xp_y,final_armor_y,final_atkdmg_y,final_health_y,final_dmgdone_y,final_dmgtaken_y,final_truedmgdone_y,final_truedmgtaken_y,final_lvl_y
0,8000,100,788,31.0,17313,18099,288,510,6304,11079,...,16933,17117,272,508,5731,12235,21465,521,604,33
1,8001,100,685,31.0,17492,16756,264,499,5584,14834,...,17533,16527,285,424,6141,14326,21925,160,176,33
2,8002,100,612,29.0,16206,16640,303,494,5869,15843,...,17480,16518,268,509,6248,17065,23649,376,338,33
3,8003,100,707,30.0,17784,16759,293,565,6250,17100,...,18126,17173,402,521,6314,16230,25839,604,1497,34
4,8004,100,611,27.0,16341,18377,267,454,5802,13477,...,16993,17113,324,523,6612,11488,20156,406,687,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,100,905,35.0,15440,17688,279,414,6072,13331,...,15060,17845,285,449,6519,11433,21529,743,1835,35
1996,9996,100,560,28.0,19123,18273,291,541,6542,13467,...,17433,18545,328,566,6238,14474,23791,76,1566,36
1997,9997,100,510,25.0,14719,17211,282,512,5914,7891,...,17129,18888,269,507,5629,14181,18033,523,394,35
1998,9998,100,556,30.0,18353,18298,344,625,6043,13838,...,15134,16823,293,495,6042,12048,23098,70,679,34


**The following code calculate the NN classification for league of leguends**

In [12]:
#y_train_timeline = y_train_timeline.sample(n=8000, random_state=345)

In [30]:
X_train = X_teams
y_train = y_train

X_test = y_teams_test

In [37]:
y_train

0       200
1       200
2       200
3       100
4       200
       ... 
7995    200
7996    100
7997    200
7998    200
7999    100
Name: winner, Length: 8000, dtype: int64

In [31]:
pipe = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(hidden_layer_sizes = (2,),
                             activation = 'tanh',
                             max_iter = 10000))
    ]
)

In [32]:
pipe.fit(X_train, y_train)    

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('nn',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(2,),
                               max_iter=10000))])

In [33]:
y_pred = pipe.predict(X_test)
y_pred

array([100, 100, 200, ..., 200, 100, 100])

In [38]:
accuracy_score(y_train, pipe.predict(X_train))


0.711

In [42]:
submission['winner'] = y_pred
submission.head()


,matchId,winner
0,8000,100
1,8001,100
2,8002,200
3,8003,100
4,8004,200


In [ ]:
#submission.to_csv('ds6-league-of-legends/submission.csv', index = False)

# Tried to fit a model with the test data 

The results are much lower. 

In [ ]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_test = pd.merge(X_test_original, Y_test_timeline, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_test = (
    X_test
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

X_test = X_test.groupby('matchId')[variables].sum().reset_index(drop = True)

**Testing something different since the predictions are low still**

# Trying a new model to test better results



In [ ]:
data = pd.json_normalize(champions['data'])
data

In [ ]:
data


In [ ]:
data['key'] = data['key'].astype(int)

variables = ['participantId','summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg', 'final_atkspd', 
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl',
             'info.attack', 'info.defense', 'stats.hpregen', 'stats.mpregen', 'stats.attackdamage', 'stats.attackspeed']

X = pd.merge(X_train_original, data, how='inner', left_on='championId', right_on='key')

X = pd.merge(X, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X = pd.merge(X, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X = X.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X = (
    X
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'participantId':'max',
        'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum',
          'stats.attackspeed': 'sum',
          'info.attack': 'sum',
          'stats.mpregen': 'sum',
          'stats.attackdamage': 'sum',
          'info.defense': 'sum',
          'stats.hpregen': 'sum',
          'final_atkspd': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

X = X.groupby('matchId')[variables].sum().reset_index(drop = True)

In [ ]:
X

In [ ]:
y = y_train_timeline

y = y.sample(n=8000, replace=True)

X = X.sample(n=8000, replace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [ ]:
X_test.shape

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

logReg_aj_scaler = Pipeline(
    steps = [
        ('impute', SimpleImputer()), #fills average of the column if there are NaN missing values
        ('scaler', StandardScaler()),
        ('log_reg', LogisticRegression(C = 0.0001)) #started from 0.01, 0,001, 
    ]
).fit(X_train,y_train)


In [ ]:
scal_pred = logReg_aj_scaler.predict(X_test)
scal_pred

In [ ]:
accuracy_score(y_test, scal_pred)

In [ ]:
print(classification_report(y_test, scal_pred))

In [ ]:
logReg_aj_scaler['scaler'].get_feature_names_out()
logReg_aj_scaler['log_reg'].coef_.flatten()


In [ ]:
pd.DataFrame({'feature': X_train.columns,
                             'coefficient': logReg_aj_scaler['log_reg'].coef_.flatten()}).sort_values('coefficient', 
                                                                                                      ascending =False).iloc[0:20]

# trying something new

In [ ]:
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold

In [ ]:
data['key'] = data['key'].astype(int)

variables = ['participantId','summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg', 'final_atkspd', 
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl',
             'info.attack', 'info.defense', 'stats.hpregen', 'stats.mpregen', 'stats.attackdamage', 'stats.attackspeed']

categorical_variables = y = submission['winner']

X_new = pd.merge(X_train_original, data, how='inner', left_on='championId', right_on='key')

X_new = pd.merge(X_new, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X_new = pd.merge(X_new, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_new = X_new.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_new = (
    X_new
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'participantId':'max',
        'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum',
          'stats.attackspeed': 'sum',
          'info.attack': 'sum',
          'stats.mpregen': 'sum',
          'stats.attackdamage': 'sum',
          'info.defense': 'sum',
          'stats.hpregen': 'sum',
          'final_atkspd': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

X_new = X_new.groupby('matchId')[variables].sum().reset_index(drop = True)

In [ ]:
#y = submission['winner']
#X = X_new[variables]

#y = y.sample(n=2000, replace=True)

#X = X.sample(n=2000, replace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

In [ ]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False, drop = 'first'), categorical_variables)
            ],
            remainder = 'passthrough', verbose_feature_names_out = False)),
        ('pf', PolynomialFeatures(interaction_only = True, include_bias = False)),
        ('vt', VarianceThreshold()),
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression())
    ]
)

pipe.fit(X_train, y_train)